In [1]:
import os
os.chdir('/Users/ioana/Desktop/Thesis/ML_HEP_Tutorials_v2/diquark-analysis_semileptonic')

import pandas as pd
from tqdm import tqdm

import sys
print(sys.path)


['/Users/ioana/Desktop/Thesis/ML_HEP_Tutorials_v2/diquark-analysis_semileptonic/notebooks', '/opt/homebrew/Cellar/python@3.11/3.11.6/Frameworks/Python.framework/Versions/3.11/lib/python311.zip', '/opt/homebrew/Cellar/python@3.11/3.11.6/Frameworks/Python.framework/Versions/3.11/lib/python3.11', '/opt/homebrew/Cellar/python@3.11/3.11.6/Frameworks/Python.framework/Versions/3.11/lib/python3.11/lib-dynload', '', '/Users/ioana/Library/Caches/pypoetry/virtualenvs/diquark-jOxHWOVb-py3.11/lib/python3.11/site-packages', '/Users/ioana/Desktop/Thesis/ML_HEP_Tutorials_v2/diquark-analysis']


In [2]:
from diquark import DATA_KEYS, PATH_DICT, CROSS_SECTION_DICT
from diquark.helpers import create_data_dict, get_col
from diquark.load import read_jet_delphes, read_met_delphes
from diquark.features import (
    jet_multiplicity,
    leading_jet_arr,
    calculate_delta_r,
    combined_invariant_mass,
    three_jet_invariant_mass,
    calculate_missing_transverse_energy,
    missing_ET,
)
from diquark.plotting import make_histogram, make_histogram_with_double_gaussian_fit


if os.getcwd().split("/")[-1] == "notebooks":
    os.chdir("..")

In [3]:
datasets = {key: read_jet_delphes(PATH_DICT[key]) for key in tqdm(DATA_KEYS)}
datasets2 = {key2: read_met_delphes(PATH_DICT[key2]) for key2 in tqdm(DATA_KEYS)}


100%|██████████| 2/2 [00:00<00:00,  8.47it/s]


In [4]:
jet_multiplicities = {key: jet_multiplicity(ds) for key, ds in tqdm(datasets.items())}

100%|██████████| 2/2 [00:00<00:00, 9436.00it/s]


In [5]:
jet_pts = {key: leading_jet_arr(data, key="Jet/Jet.PT") for key, data in tqdm(datasets.items())}
jet_etas = {key: leading_jet_arr(data, key="Jet/Jet.Eta") for key, data in tqdm(datasets.items())}
jet_phis = {key: leading_jet_arr(data, key="Jet/Jet.Phi") for key, data in tqdm(datasets.items())}

100%|██████████| 2/2 [00:00<00:00, 726.48it/s]


In [7]:
miss_MET = {key: missing_ET(data2, key="MissingET/MissingET.MET") for key, data2 in tqdm(datasets2.items())}

100%|██████████| 2/2 [00:00<00:00, 546.74it/s]


In [9]:
missing_energy = {} #key: calculate_missing_transverse_energy(px_miss, py_miss) for key, arr in tqdm(datasets.items())
for key2, data2 in tqdm(datasets2.items()):
    px_miss = missing_ET(data2, 4, key="MissingET/MissingET.MET")
    py_miss = missing_ET(data2, 4, key="MissingET/MissingET.MET")
    
    missing_energy[key2] = calculate_missing_transverse_energy(px_miss, py_miss)


100%|██████████| 2/2 [00:00<00:00, 363.30it/s]


In [10]:
combined_masses = {key: combined_invariant_mass(arr) for key, arr in tqdm(datasets.items())}

/Users/ioana/Desktop/Thesis/ML_HEP_Tutorials_v2/diquark-analysis_semileptonic/diquark/features.py:135: RuntimeWarning: invalid value encountered in sqrt
  return np.nan_to_num(np.sqrt(E_total**2 - px_total**2 - py_total**2 - pz_total**2))
100%|██████████| 2/2 [00:00<00:00, 65.13it/s]


In [12]:
delta_rs = {}
for key, data in tqdm(datasets.items()):
    etas = leading_jet_arr(data, 4, key="Jet/Jet.Eta")
    phis = leading_jet_arr(data, 4, key="Jet/Jet.Phi")
    pts = leading_jet_arr(data, 4, key="Jet/Jet.PT")

    delta_rs[key] = calculate_delta_r(etas, phis, pts)

100%|██████████| 2/2 [00:00<00:00, 65.92it/s]


In [13]:
m3j_s = {}
for key, data in tqdm(datasets.items()):
    m3j_s[key] = three_jet_invariant_mass(data)

100%|██████████| 2/2 [00:01<00:00,  1.01it/s]


In [14]:
fig = make_histogram(combined_masses, 20, clip_top_prc=100)
bin_width = fig.data[0].x[1] - fig.data[0].x[0]
fig.update_layout(
    title="6-jet Mass",
    xaxis_title="Invariant Mass [GeV]",
    yaxis_title_text="count x sigma",
    barmode="stack",
    bargap=0,
    width=1600 * (2 / 3),
    height=900 * (2 / 3),
    # ignore first bin
    xaxis_range=[fig.data[0].x[1] - bin_width / 2, fig.data[0].x[-1] + bin_width / 2],
    yaxis_type="log",
)

fig.show()
print(
    [fig.data[0].x[1] - bin_width, fig.data[0].x[-1]],
)

[0.0, 9417.129991671465]


In [15]:
ds = create_data_dict(
    **{
        "multiplicity": jet_multiplicities,
        "delta_R": delta_rs,
        "m3j": m3j_s,
        "inv_mass": combined_masses,
        "pt": jet_pts,
        "eta": jet_etas,
        "phi": jet_phis,
        "mET": missing_energy,
    }
)

In [16]:
df = pd.DataFrame(ds)
df["target"] = df["Truth"].apply(lambda x: 1 if "SIG" in x else 0)

In [17]:
df.head()

,Truth,multiplicity,delta_R_1,delta_R_2,delta_R_3,delta_R_4,delta_R_5,delta_R_6,m3j_1,m3j_2,...,phi_2,phi_3,phi_4,phi_5,phi_6,mET_1,mET_2,mET_3,mET_4,target
0,BKG:semilepto_test,7,0.0,1.616195,2.158732,2.158483,0.0,0.0,2912.781428,2891.540087,...,2.587464,2.895854,-0.808096,-1.254239,0.474848,1627.102595,0.0,0.0,0.0,0
1,BKG:semilepto_test,7,0.0,2.794346,3.630165,2.483253,0.0,0.0,1911.636997,1450.750761,...,-2.978855,2.746671,2.002219,-1.144244,2.155058,846.806205,0.0,0.0,0.0,0
2,BKG:semilepto_test,6,0.0,3.095241,2.633242,2.163251,0.0,0.0,3395.286840,2636.057382,...,-0.625289,-1.476933,-1.581123,-2.121839,-0.053876,233.956490,0.0,0.0,0.0,0
3,BKG:semilepto_test,5,0.0,1.476971,1.050706,3.225524,0.0,0.0,2540.088762,2034.811872,...,-2.573591,-3.015218,-1.501568,-0.330397,0.000000,2224.845368,0.0,0.0,0.0,0
4,BKG:semilepto_test,5,0.0,3.084862,1.971243,3.002193,0.0,0.0,3638.339959,3222.440977,...,-2.473694,2.331247,-2.901099,2.750091,0.000000,2110.220183,0.0,0.0,0.0,0


In [18]:
df.to_parquet("data/full_sample.parquet", index=False)

# Data Visualization

In [20]:
fig = make_histogram(jet_pts, 20, col=0, clip_top_prc=100)
bin_width = fig.data[0].x[1] - fig.data[0].x[0]
fig.update_layout(
    title="4-jet Mass",
    xaxis_title="Invariant Mass [GeV]",
    yaxis_title_text="count x sigma",
    barmode="stack",
    bargap=0,
    width=1600 * (2 / 3),
    height=900 * (2 / 3),
    # ignore first bin
    xaxis_range=[fig.data[0].x[1] - bin_width / 2, fig.data[0].x[-1] + bin_width / 2],
    yaxis_type="log",
)

fig.show()

In [21]:
suu_mass = {"SIG:suu": combined_masses["SIG:suu"]}
fig = make_histogram_with_double_gaussian_fit(suu_mass, 20, clip_top_prc=100, cross=None)
bin_width = fig.data[0].x[1] - fig.data[0].x[0]
fig.update_layout(
    title="4-jet Mass",
    xaxis_title="Invariant Mass [GeV]",
    yaxis_title_text="probability density",
    barmode="stack",
    bargap=0,
    width=1300 * (2 / 3),
    height=1300 * (2 / 3),
    # ignore first bin
    xaxis_range=[fig.data[0].x[1] - bin_width / 2, fig.data[0].x[-1] + bin_width / 2],
    yaxis_type="log",
)
fig.update_legends(
    title_text="",
    itemsizing="constant",
    yanchor="top",
    y=0.1,
    xanchor="left",
    x=0.01,
    font=dict(size=16),
)
fig.show()
# fig.write_image("suu_mass.pdf")

  0%|          | 0/2 [02:40<?, ?it/s]


RuntimeError: Optimal parameters not found: Number of calls to function has reached maxfev = 800.